In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 流式调用

In [6]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo:free",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    # 启用流式调用
    streaming=True,
)

for chunk in llm.stream("写一首关于冬天的诗歌"):
    print(chunk.content,end="",flush=False)

冬天来了，白雪飘飘，
寒风呼啸，树叶凋零。
寒冷的夜晚，火炉闪烁，
取暖的火光，温暖了心。

冰冷的湖面，冰花绽放，
雪花飘飞，铺满大地。
孩子们欢笑，雪人作伴，
快乐的时光，刻在心间。

冬天来了，万物沉睡，
寂静的夜晚，星空闪烁。
月光如水，洒在身旁，
温柔的光芒，抚平疲惫。

冬天来了，寒意逼人，
温暖的家园，温情满溢。
团圆的时刻，永不忘记，
冬天的温暖，永远在心。

# 追踪Token的使用

In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo:free",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲个冷笑话")
    print(result)
    print(cb)

content='为什么猫咪喜欢打架？\n\n因为它们是“拳王”！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 16, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ccr0NcRS6DV2qTbmG9DtYGi3DRO5w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--f04e7865-111d-4989-8f19-699ea78d8d1c-0' usage_metadata={'input_tokens': 16, 'output_tokens': 31, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Tokens Used: 47
	Prompt Tokens: 16
		Prompt Tokens Cached: 0
	Completion Tokens: 31
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $5.4499999999999997e-05


# 自定义输出
- 输出函数参数
- 输出json
- 输出List
- 输出日期

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import  PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, field_validator, Field
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo:free",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0,
)

# 定义最终返回的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")

    # 校验问题是否符合要求
    @field_validator("setup")
    def question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("问题必须以问号结尾")
        return field

parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

prompt_and_model = prompt | llm
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})
print("out_put:",out_put)


out_put: content='{\n  "setup": "为什么程序员总是冷静？",\n  "punchline": "因为他们有很多if条件！"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 214, 'total_tokens': 254, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CcrGpULG2Gx6obLAQx6ZyeI7yYiqY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--4643a937-df3c-4d8e-a466-9568b4984854-0' usage_metadata={'input_tokens': 214, 'output_tokens': 40, 'total_tokens': 254, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
# 输出成list的形式
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import  CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, field_validator, Field
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo:free",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0,
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)
_input = prompt.format(subject="水果")
out_put = llm.invoke(_input)
print("out_put:",out_put)
parser.parse(out_put.content)

out_put: content='apple, banana, orange, strawberry, pineapple' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 42, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Ccu02JzpneVeBhmztphftEZ2nC08Z', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--4d72b161-e915-4c74-9b80-430d5541ae4d-0' usage_metadata={'input_tokens': 42, 'output_tokens': 9, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


['apple', 'banana', 'orange', 'strawberry', 'pineapple']